## Train an image classifier from scratch

In this network we will look at how to train basic image classifier from scratch in PyTorch.

**Before you go any further** follow the instructions in the file `create-classification-dataset.ipynb` to make you dataset before you can train anything here. 

Work your way through this notebook, running each cell and looking carefully at what the code is doing. There are some tasks at the bottom of the notebook. 

Once you run this code, you can compare it with the code in the notebook `train-image-classifier-from-pretrained.pynb`. 

First lets do some imports:


In [ ]:
import torch
import torchvision

import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.utils as vutils
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

If you are getting an error you may need to uncomment the next line to install sklearn:

In [ ]:
#!pip install scikit-learn

##### Hyperparameters

Now lets define our hyperparameters. **If your dataset has more than 3 classes** make sure to change the parameter `num_classes`!

In [ ]:
device = 'cpu'
momentum = 0.9
num_epochs = 10
num_classes = 3
val_size = 0.3
batch_size = 100
learn_rate = 0.001
data_path = '../data/my-data/my-classification-dataset/'

##### Training image transforms

Here we define our image transforms (and data augmentation) for our training data. Once you have started to train your model, come back to this cell for task 2.

In [ ]:
train_transform = transforms.Compose(
    [
        #
        transforms.RandomHorizontalFlip(p=0.5),
        #
        transforms.RandomAffine(degrees=(-30,30),translate=(0.25,0.25),scale=(0.85,1.15)),
        #
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
        #
        transforms.RandomResizedCrop(size=(64, 64), antialias=True),
        #
        transforms.ToTensor(),
        #
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

##### Validation image transforms

Here we define our image transforms for our validation data. How do they differ from the training transforms?

In [ ]:
val_transform = transforms.Compose(
    [   
        torchvision.transforms.Resize(64, antialias=True),
        torchvision.transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

##### Create datasets

Here we create our dataset classes. Because we are using different transforms, we need to make two seperate dataset classes. We will then take a random sub-selection of our data and split our dataset into two. 

When we do the split, by setting `random_state=42`, we are doing this in a deterministic way, such that we will always get the same 'random' split of data into the training and validation sets.

In [ ]:
# Instatiate train and validation dataset with seperate transforms
train_dataset = ImageFolder(data_path, transform=train_transform)
val_dataset = ImageFolder(data_path, transform=val_transform)

# Get length of dataset and indicies
num_train = len(train_dataset)
indices = list(range(num_train))

# Get train / val split for data points
train_indices, val_indices = train_test_split(indices, test_size=val_size, random_state=42)

# Override dataset classes to only be samples for each split
train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
val_dataset = torch.utils.data.Subset(val_dataset, val_indices)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

##### Plot training images

Here we are plotting a sample of training images. See how the data augmentation transforms are manipulating the images, compared to the images in the validation set (visualised in the next cell).

In [ ]:
# Plot some training images
real_batch = next(iter(train_loader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

##### Plot validation set images

In [ ]:
# Plot some training images
real_batch = next(iter(val_loader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Validation Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

##### Define convolutional neural network

Here we define our convolutional neural network. Once you have started to train your model, come back to this cell for task 3.

In [ ]:
#  Here we define our class for our CNN based classifier
#  This model takes images and input and gives a prediction over the classes provided in the training data
class ClassificationNetwork(nn.Module):
    # Constructor for the class
    def __init__(self):
        # Call the constructor of the base class nn.module
        super().__init__()
        # First convolutional layer
        # Takes a 3D tensor with 3 channels as input (3 colour channels in the image)
        # Outputs a 3D tensor with 64 channels (outputs of convolutional filters)
        # The convolutional kernel size is 5x5 
        self.conv1 = nn.Conv2d(3, 64, 5)
        # Second convolutional layer
        # Takes a 3D tensor with 64 channels as input 
        # Outputs a 3D tensor with 128 channels (outputs of convolutional filters)
        # The convolutional kernel size is 5x5 
        self.conv2 = nn.Conv2d(64, 128, 5)
        # Third convolutional layer
        # Takes a 3D tensor with 128 channels as input 
        # Outputs a 3D tensor with 128 channels 
        # The convolutional kernel size is 5x5 
        self.conv3 = nn.Conv2d(128, 128, 5)
        # Max-pooling function 
        # This is reused through the network in the forward pass
        self.pool = nn.MaxPool2d(2, 2)
        # First fully connected layer
        # Takes a vector of 2048 as input and outputs a vector of 256
        self.fc1 = nn.Linear(2048, 256)
        # Second fully connected layer
        # Takes a vector of 256 as input and outputs a vector of class predictions
        self.fc2 = nn.Linear(256, num_classes)

    # Definition of the forward pass
    # Here the classifier takes an image as input and predicts an vector of probabilites, one for each class in  our dataset
    def forward(self, x):
        # Pass input through first convolutional layer with relu activation function
        x = F.relu(self.conv1(x))
        # Use max-pooling to downsample the output of the first conv layer
        x = self.pool(x)
        # Pass output of maxpooling through second convolutional layer with relu activation function
        x = F.relu(self.conv2(x))
        # Use max-pooling to downsample the output of the first conv layer
        x = self.pool(x)
        # Pass output of maxpooling through third convolutional layer with relu activation function
        x = F.relu(self.conv3(x))
        # Use max-pooling to downsample the output of the first conv layer
        x = self.pool(x)
        # Flatten the output of the last maxpooling operation into a vector of size 2048
        x = torch.flatten(x, 1)
        # Pass vector through first fully connected layer with relu activation function
        x = F.relu(self.fc1(x))
        # Pass vector through second fully connected layer (no activation function needed here)
        x = self.fc2(x)
        # Output vector of class probabilities
        return x

##### Setup core objects

Here we setup our core objects, the model, the loss function and the optimiser.

In [ ]:
model = ClassificationNetwork()
model.to(device)

# Cross entropy loss for training classification
criterion = nn.CrossEntropyLoss()

# Stochastic gradient descent loss
optimizer = optim.SGD(model.parameters(), lr=learn_rate, momentum=momentum)

##### Training loop

Here is our training loop for our data. Look at how the training dataset and validation dataset are used? 

What differences are there in the code when we cycle through each of these sets of data?

In [ ]:
train_losses = []
val_losses = []

best_loss = 100000
for epoch in range(num_epochs): 
    train_loss = 0.0
    
    # Training loop
    for i, data in enumerate(train_loader, 0):
        # Get data
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Process data
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        
        # Update model weights
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    
    # Validation loop
    with torch.no_grad():
        val_loss = 0.0
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    # Normalise cumulative losses to dataset size
    train_loss = train_loss / len(train_loader)
    val_loss = val_loss / len(val_loader)
    
    # Added cumulative losses to lists for later display
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    print(f'Epoch {epoch + 1}, train loss: {train_loss:.3f}, val loss: {val_loss:.3f}')
    
    # if validation score is lowest so far, save the model
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'best_model_from_scratch.pt')

##### Plot training vs validation loss 

Lets plot our training vs validation loss over time. If you train for long enough you will usually see the validation loss start to get worse at some point while the training loss will continue to get better. This occurs when the model starts to **overfit** to the training data, and become worse at accurately classifying unseen data. 

When our model is giving the best performance on our validation data (before the validation loss starts to increase) is when we would perform early stopping. If you haven't observed that here, you may need to re-run this code and train for more epochs. Given the limited time available in class, you may want to do this at home, running the code overnight (with your laptop plugged in!)

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Train vs validation loss")
plt.plot(train_losses,label="train")
plt.plot(val_losses,label="val")
plt.xlabel("epochs")
plt.ylabel("cumulative loss")
plt.legend()
plt.show()

### Tasks

**Task 1:** Run all the cells in this code to train a classifier on your custom dataset. While this is training, you can do tasks 2 and 3.

**Task 2:** Go to [the cell that defines the training transformations](#training-image-transforms) and look at that code. Write a comment for every line to describe what the code is doing. If you are unsure you can consult [the pytorch reference](https://pytorch.org/vision/0.9/transforms.html).

**Task 3:** Go to [the cell that defines the convolutional neural network](#define-convolutional-neural-network) and look at that code. Based on the material from the lecture try to write a comment for each line of code in this cell. Consult [the pytorch neural network (torch.nn) reference](https://pytorch.org/docs/stable/nn.html) and [the W3 schools reference on python inheritance](https://www.w3schools.com/python/python_inheritance.asp) for any code that you are unsure about. 

**Task 4:** Go the to the notebook `train-image-classifier-from-pretrained.ipynb` and run the code in there to train a model. Compare the code in this notebook to that? Where does the code differ? Compare the losses in training, which approach was quicker to converge onto an accurate model?
